# 6. Numerical scheme for evaluating Dawson functions
## Definition  
Define the following family of Dawson-like functions  
$g(x)=e^{x^2}\int_{-\infty}^x e^{-u^2}du$  
$h(x)=e^{x^2}\int_{-\infty}^x e^{-u^2}[g(u)]^2du$  
$G(x)=\int_0^x g(u)du$  
$H(x)=\int_{-\infty}^x h(u)du$  

## Motivation
In Moment Neural Network (MNN), the mean and the variance of the inter-spike-intervals (ISI) are defined through above integrals as follows  
$\mathbb{E}[T]\propto \int_a^b g(t)dt$,  
and  
${\rm Var}[T]\propto \int_a^b h(t)dt$.  
The evaluation of these integrals become extremely problematic for two reasons:  
- Reliability: for $x<-1$, direct numerical evaluation of $g(x)$ and $h(x)$ using the definition encounters '$\infty\cdot0$' type numeric instability, even for moderately large value of $|x|$. This scenario is frequently encountered as negative values of $x$ happen to be in the biological range.  
- Speed: direct evaluation of the mean and variance of ISI involves double or triple integrals that are costly to compute. A temporary workaround is to interpolate with a pre-calculated lookup table, however, the range and precision of this method is limited.

These issues severely hinder the ability to scale up MNN and future research aimed at using MNN to perform deep learning.


## Overview of the numerical scheme
To resolve these issues, we employ a combination of techniques to approximate the Dawson functions. The overall strategy of the numeric scheme is to break the domain of these functions into four regions:  
- i. Extremely negative values  $A=(-\infty,-c]$  
- ii. Intermediate negative values  $B=(-c,0]$   
- iii. Intermediate positive values $C=(0,c]$    
- iv. Extremely positive values $D=(c,\infty)$    

where $c$ is a sufficiently large positive constant.  
For region $A$ and $D$, the functions are approximated using their respective asymptotic expansions.  
For region $B$ and $C$ the functions are approximated using Chebyshev polynomials (after a suitable transformation).  
This idea is inspired by the established methods for evaluating the scaled complementary error function.  

## Approximation by asymptotic expansions  
We tabulate the asymptotic expansions as below  
\begin{array}{|c|c|}
\hline & x\to +\infty & x\to -\infty \\\hline
  g(x)  & \sqrt{\pi}e^{x^2}-\frac{1}{2\sqrt{\pi}}\displaystyle\sum_{n=0}^\infty(-1)^n\Gamma(\tfrac{1}{2}+n)\dfrac{1}{x^{2n+1}} & -\frac{1}{2\sqrt{\pi}}\displaystyle\sum_{n=0}^\infty(-1)^n\Gamma(\tfrac{1}{2}+n)\dfrac{1}{x^{2n+1}}\\\hline
  G(x)  & \frac{1}{2}e^{x^2}\displaystyle\sum_{n=0}^\infty\Gamma(\tfrac{1}{2}+n)\dfrac{1}{x^{2n+1}} & \frac{1}{4}\phi_0(\frac{1}{2}) - \frac{1}{2}\log(-x) -\frac{1}{8x^2}+\frac{3}{32x^4}-\frac{5}{32x^6} +\dots \\\hline
  h(x)  & \left(\frac{\sqrt{\pi}}{2}\right)^3e^{x^2}[{\rm erfc}(-x)]^2{\rm erfi}(x) & \displaystyle\sum_{n=0}^\infty \dfrac{a_n}{x^{2n+3}} \\\hline
  H(x)  & \left(\frac{\pi^2}{32}\right)[{\rm erfi}(x)-1][{\rm erfc}(-x)]^2{\rm erfi}(x) & \displaystyle\sum_{n=0}^\infty \dfrac{-a_n}{2n+2}\dfrac{1}{x^{2n+2}} \\\hline  
\end{array}

Remark. The function $g(x)$ is related to the scaled complementary error function as

$$g(x) = \tfrac{\sqrt{\pi}}{2}{\rm erfcx(-x)}$$  

whose asymptotic expansions are well known. The results for $G(x)$, $h(x)$, and $H(x)$ are new.  

NB. It appears that the expansion for $G(x)$ as $x\to+\infty$ has an extra non-vanishing term that is equal to $G(-x)$!

## Chebyshev polynomial approximation for the intermediate range  


### i. For negative values of $x$  
Since the Dawson functions decay to zero rapidly as $x\to-\infty$, fitting them directly with Chebyshev polynomials results in poor convergence, and suitable transformations are required.

First, we map the input $x\in(-\infty,0]$ to the unit interval with $x'=\dfrac{4}{4-x}$ to obtain  
$\tilde{g}(x') = g(x)$  
$\tilde{h}(x') = h(x)$  
$\tilde{H}(x') = H(x)$  

Next, we subdivide $x'\in(0,1]$ into $N$ equal intervals and fit the transformed functions over each interval using Chebyshev polynomials of degree $K$. For reference we list the first few terms of the Chebyshev basis    
$T_0(x) = 1$  
$T_1(x) = x$  
$T_2(x) = 2x^2-1$  
$T_3(x) = 4x^3-3x$

This procedure results in a matrix of size $(N,K+1)$ whose entries are the Chebyshev coefficients, which are then used to approximate the original function. Alternatively, we can convert the coefficients into the standard polynomial basis which may be more convenient for numerical implementation (especially for GPU).

### ii. For positive values of $x$  
For $g(x)$ and $G(x)$ we can use the special identities:  
- $g(x)=\sqrt{\pi}e^{x^2}-g(-x)$  
- $G(x)=\frac{\pi}{2}{\rm erfi}(x)+G(-x)$  
- $h(x) = \sqrt{\pi}e^{x^2}\left[\tfrac{1}{2}\log2+G(x)+G(-x)\right] - h(-x)$


Proof.  
For $g(x)$ and $G(x)$ the proofs are trivial.  
For $h(x)$ consider the following  
$h(x)-h(-x) = e^{x^2}\left[\int_{-\infty}^x e^{-t^2}g(t)^2dt - \int_{-\infty}^{-x} e^{-t^2}g(t)^2dt\right]$  
$= e^{x^2}\left[\int_{-x}^0 e^{-t^2}g(t)^2dt+\int_{0}^x e^{-t^2}g(t)^2dt\right]$  
$= e^{x^2}\left(\int_{0}^x e^{-t^2}[g(t)^2+g(-t)^2]dt\right)$  

Substitute in the identity $g(x)=\sqrt{\pi}e^{x^2}-g(-x)$ we get  
$g(t)^2 + g(-t)^2 = 2g(t)^2 -2\sqrt{\pi}e^{t^2}g(t) + \pi e^{2t^2}$  
Integrating both sides of the equation from $t=0$ to $t=x$ leads to    
$h(x)-h(-x) = 2[h(x)-e^{x^2}h(0)] -2\sqrt{\pi}e^{x^2}G(x) + \frac{\pi\sqrt{\pi}}{2}e^{x^2}{\rm erfi}(x)$  
where $h(0) = \frac{\sqrt{\pi}}{4}\log 2$.  
Rearrange we get  
$h(x)+h(-x) = e^{x^2}\left[2h(0) +2\sqrt{\pi}G(x) - \frac{\pi\sqrt{\pi}}{2}{\rm erfi}(x)\right]$  
Or alternatively  
$h(x)+h(-x) = \sqrt{\pi}e^{x^2}\left[\tfrac{2}{\sqrt{\pi}}h(0) +G(x) + G(-x)\right]$  

For $H(x)$ consider the following  
$H(x)-H(-x) = \int_0^x[h(t)+h(-t)]dt$  
Integrate term-by-term  
- $\tfrac{\sqrt{\pi}}{2}\log2\int_0^x e^{t^2}dt=\tfrac{\pi}{4}(\log2){\rm erfi}(x)$  
- $2\sqrt{\pi}\int_0^x e^{t^2}G(t)dt = \pi{\rm erfi}(x)G(x) - \frac{\pi\sqrt{\pi}}{2}\int_0^x{\rm erfi}(t)e^{t^2}{\rm erfc}(-t)dt$  
- $-\tfrac{\pi^2}{8}[{\rm erfi}(x)]^2$  

It is virtually impossible to cast the integral in simple forms and an alternative method is required.  
$\square$

Alternative strategy. Note the following identity:  
$2\sqrt{\pi}G(x) - \frac{\pi\sqrt{\pi}}{2}{\rm erfi}(x) = \pi\int_0^x e^{t^2}{\rm erf}(t)dt
=\sqrt{\pi} x^2 {_2F_2(1,1;\tfrac{3}{2},2|x^2)}$  
where $_pF_q$ is the generalized hypergeometric function. This result by itself is not useful for numeric implementation but provide some opportunities for further results.  Consider the integral  
$J(x) = \pi\int_0^x\int_0^u e^{u^2+t^2}{\rm erf}(t)dtdu$  
The region of integration is a triangle on the $(u,t)$ plane with $u\in(0,x)$ and $t\in(0,u)$. Now transform it to polar coordinates $(\theta,r)$, we have  
$u^2+t^2 = r^2$  
$t = r\sin\theta$  
$dtdu = rdrd\theta$  
and the integration limits become $r\in(0,x\sec\theta)$ and $\theta\in(0,\tfrac{\pi}{4})$

Therefore  
$J(x) = \pi\int_0^{\tfrac{\pi}{4}}\int_0^{x\sec\theta}e^{r^2}{\rm erf}(r\sin\theta)rdrd\theta$

Use the following result  
$\int r e^{r^2}{\rm erf}(a r)dr = \tfrac{1}{2}e^{r^2}{\rm erf}(a r)
-\tfrac{1}{2}\frac{a}{\sqrt{1-a^2}}{\rm erfi}(\sqrt{1-a^2}r)$

We obtain  
$J(x) = \frac{\pi}{2}\int_0^{\tfrac{\pi}{4}} \left[ 
e^{x^2\sec^2\theta}{\rm erf}(x\tan\theta) - \tan\theta{\rm erfi}(x)
\right]d\theta$

The second term is  
$-\tfrac{\pi}{2}{\rm erfi}(x)\int_0^{\tfrac{\pi}{4}}\tan\theta d\theta = -\tfrac{\pi}{4}{\rm erfi}(x)\log2$  

For the last term that remains, let $t=\tan\theta$, thus $dt = \sec^2\theta d\theta = (1+t^2)d\theta$ and  
$H(x)-H(-x)=\tfrac{\pi^2}{4}(1-\log2){\rm erfi}(x)+\tfrac{\pi^2}{2}e^{x^2}\int_0^1e^{x^2t^2}{\rm erf}(xt)\frac{1}{1+t^2}dt$  
The integration limit is mapped from $\theta\in[0,\tfrac{\pi}{4}]$ to $t\in[0,1]$.  
This integral cannot be further reduced and its usefulness is limited.  


Need to check if this result is correct!  
Evaluate with quadrature? Or not => as $x$ becomes large the integrand becomes extremely curved!

Alternatively, use integration by parts we can get  
$H(x)-H(-x) = \tfrac{\pi}{2}{\rm erfi}(x)[\tfrac{1}{2}\log2+G(t)+G(-t)] - \pi\int_0^x {\rm erfi}(t)e^{t^2}{\rm erf}(t)dt$  
Unfortunately, the second term is intractable.

This is hopeless. Proceed ahead don't waste more time.  
Already done: g(x) is done. h(x) is done.  

1. Implement H(x) for x < 0 [done!]
2. No need to rebuild atm. Make it work first then improve speed with version 3.
3. Improve G(x) for x < 0 (check how well it works now) and implement G(x) > 0
4. Need to find new way to do H(x) for x > 0 [done! with basic method = subdivision + chebyshev; not ideal since the fit is steep but it works.]